# ![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

# Procesamiento por lotes (batch) vs. interactivo (streaming)

Apache Spark incluyó en su versión 2.0 la primera versión de una nueva API para el procesamiento en flujo de "nivel superior", en inglés la denominó "Structured Streaming". En esta PEC veremos como usar dicha API sobre Spark DataFrame para construir aplicaciones de "Flujo Estructurado". Nuestro objetivo será calcular métricas en tiempo real, como conteos, o promedios en tiempo real dentro de ventanas (p.ej. _moving average_) en una secuencia de acciones con marca de tiempo (p.ej. acciones Abrir y Cerrar en nuestros datos de muestra).

** Esta PEC cubrirá: **
* *Parte 1: Conocimiento del dominio*
* *Parte 2: Procesamiento por lotes* (3 puntos sobre 10)
* *Parte 3: Procesamiento interactivo* (7 puntos sobre 10)

#### Parte 1. Datos de esta PEC

Podemos encontrar algunos ejemplos de datos en flujo en los archivos ubicados en ```/databricks-datasets/structured-stream/events/```. Estos datos son los que vamos a usar para construir las diferentes métricas. Veamos que contiene este directorio ejecutando la siguiente celda.

In [3]:
%fs ls /databricks-datasets/structured-streaming/events/

Hay aproximadamente unos 50 archivos JSON. Veamos que contiene uno de ellos, por ejemplo el archivo ```file-0.json```

In [5]:
%fs head /databricks-datasets/structured-streaming/events/file-0.json

Cada linea del archivo contiene un registro JSON con dos campos: ```tiempo``` y ```acción```. Tratemos de analizar estos archivos primero como si fueran ficheros en lote y luego de forma interactiva.

#### Parte 2. Procesamiento por lotes

El primer paso habitual para intentar procesar los datos es consultar los mismos de forma estática. Definamos para ello un DataFrame basado en el formato de los archivos y guardemos dicho DataFrame en formato de tabla.

En esta PEC no introduciremos aún como funcionan los tipos en pySpark. Esto lo haremos durante las siguientes PEC. Igualmente, para entender que estamos haciendo en la siguiente celda podemos consultar la lista completa de tipos se encuetra en el módulo [pyspark.sql.types](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#module-pyspark.sql.types). Para nuestros datos, usaremos los tipos [TimestampType()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.TimestampType) y [StringType()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.StringType).

In [8]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"

# Dado que ya hemos analizado un poco los datos y conocemos su formato, definiremos el esquema para acelerar el procesamiento (no es necesario que Spark intente inferir su esquema)
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

# Static DataFrame que representa datos en los archivos JSON
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(inputPath)
)

# Esta instruccion se ocupa de almacenar el DataFrame como una tabla de SparkQL, así podremos accederla usando lenguaje SQL
staticInputDF.createOrReplaceTempView("static_input")

display(staticInputDF)

Antes de empezar a trabajar con estos datos, reduciremos su granularidad temporal a nivel de minuto para cada tipo de acción. Además, generaremos una vista para poder usar consultas SQL y así poder calcular nuestras métricas de forma sencilla.

Para realizar este proceso, ejecutaremos la siguiente celda.

In [10]:
from pyspark.sql.functions import *      # para poder usar la funcion window()

staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.action, 
       window(staticInputDF.time, "1 minute"))    
    .count()
)
staticCountsDF.cache()

# Registrar el DataFrame como una tabla llamada 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

display(staticCountsDF)

#### Ejercicio 2(a)

Modifica la granularidad del dataframe ```staticCountsDF``` a nivel de hora y repite el mismo conteo. Para superar el test, la columna de salida del dataFrame ha de conservar el mismo nombre de columna que en el dataFrame ```staticCountsDF```.

*Hint*: Puedes usar la opción ```.withColumnRenamed("original_name", "desired_name")``` de la operación groupBy() para cambiar el nombre de las columnas del dataFrame.

In [12]:
staticCountsHourlyDF = (
  <FILL IN>
)

# Registrar el DataFrame como una tabla llamada 'static_mean'
staticCountsHourlyDF.createOrReplaceTempView("static_counts_hourly")

display(staticCountsHourlyDF)

In [13]:
#Test
from databricks_test_helper import *

Test.assertEquals(spark.sql("select max(count) from static_counts_hourly").rdd.flatMap(list).first(), 1036, "Incorrect couting by hour")
Test.assertEquals(spark.sql("select min(count) from static_counts_hourly").rdd.flatMap(list).first(), 11, "Incorrect couting by hour")

### Ejercicio 2(b)

Ahora que hemos registrado la vista ```static_counts_hourly``` usando el dataframe ```static_counts_hourly```, calcula usando una consulta SQL el número de acciones totales de cada tipo (```Open```, ```Close```).

**IMPORTANTE**: Recuerda usar ```as``` para renombrar la columna con la suma como ```total_count```.

In [15]:
sum_static_counts_hourly = spark.sql(<FILL IN>)

sum_static_counts_hourly.show()

In [16]:
Test.assertEquals(sum_static_counts_hourly.take(1)[0].asDict()['total_count'], 50000, "Incorrect total counting")

### Ejercicio 2(c)

Ahora vamos a complicar un poco el ejercicio. Cuenta el numero de acciones totales por minuto y tipo.

**IMPORTANTE**: Recuerda usar el dataframe ```static_counts```

In [18]:
window_static_counts_minute = <FILL IN>

window_static_counts_minute.show()

In [19]:
Test.assertEquals(window_static_counts_minute.take(3)[2].asDict()['count'], 11, "Incorrect counting")

Test.assertEquals(window_static_counts_minute.count(),6122,"Incorrect number of minutes")

### Ejercicio 2(d)

Ahora que ya somos capaces de contar de varias formas y con diferentes granularidades, vamos a calcular algun estadístico muy simple, como por ejemplo la media. 

Usando un código parecido al del _ejercicio 2(a)_, calcula el promedio de acciones por minuto para cada hora, independientemente si son acciones ```Open``` o ```Close```. 

**IMPORTANTE**: Recuerda renombrar la columna donde calculas la media como ```average```.

In [21]:
StaticAverageHourlyDF = <FILL IN>

# Registrar el DataFrame como una tabla llamada 'static_mean'
StaticAverageHourlyDF.createOrReplaceTempView("Static_Average_Hourly")

display(StaticAverageHourlyDF)

In [22]:
Test.assertEquals(StaticAverageHourlyDF.take(2)[1].asDict()['average'], 16.575, "Incorrect averaging")
Test.assertEquals(StaticAverageHourlyDF.take(3)[2].asDict()['average'], 16.725, "Incorrect averaging")

Test.assertEquals(StaticAverageHourlyDF.count(),53,"Incorrect number of minutes")

### Ejercicio 2(e)

Para concluir nuestros cálculos en batch, determina la hora en la que se han producido un mayor número de acciones promedio por minuto.

In [24]:
max_static_averages_hourly = <FILL IN>

¿Qué hora ha sido la que ha tenido una mayor actividad promedio por minuto?

#### Parte 3: Procesamiento interactivo

Ahora que hemos analizado los datos de forma estática, vamos a cambiar el análisis a una consulta que se actualice continuamente a medida que llegan nuevos datos. Como solo tenemos un conjunto estático de archivos, vamos a emular un flujo leyendo un archivo a la vez, en el orden cronológico en que fueron creados. La consulta que tenemos que escribir es prácticamente la misma que la anterior.

In [27]:
from pyspark.sql.functions import *

# Parecido a la definicion staticInputDF anterior, solo hemos cambiado `readStream` en lugar de `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Instanciamos el esquema de datos en formato JSON
    .option("maxFilesPerTrigger", 1)  # Trataremos los archivos como si fueran una secuencia, seleccionando un archivo a la vez
    .json(inputPath)
)

# Misma consulta que en el caso staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 minute"))
    .count()
)

Vamos a comprobar que realmente disponemos de un stream de datos

In [29]:
streamingCountsDF.isStreaming

Ahora vamos a establecer la configuración en el cluster del flujo de datos.

In [31]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # mantenemos pequeno el tamaño de los shuffle

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table
    .queryName("counts")     # counts = nombre de la tabala in-memory
    .outputMode("complete")  # complete = todos los contadores deben guardarse en la tabla
    .start()
)

### Ejercicio 3(a)

Por simplicidad en esta parte podemos usar la siguiente notación:

`%sql` que es una sentencia que solo funciona en los notebooks de Databricks. Esta ```magic function``` ejecuta `sqlContext.sql()` y pasa los resultados a la función `display()`. Estas dos sentencias son equivalentes:

`%sql select * from counts order by window`

`display(sqlContext.sql("select * from counts  order by window"))`

**Nota:** Como el comando display se ejecuta en el navegador no en el cluster, está limitado a solo mostrar las 1000 primeras filas, para contar cuantas filas se han leído del stream podéis ejecutar el siguiente código:

`%sql select count(*) from counts`

In [33]:
<FILL IN>

En la celda de arriba, además, puedes cambiar la forma de visualizar los datos, ex. En forma de tabla, histograma, linea, etc.

Visualiza en forma de histograma los resultados y re-ejecuta unas cuantas veces la celda. Podrás observar que conforme van llegando nuevos datos, la gráfica se va actualizando.

### Ejercicio 3(b)

Vamos a crear un sistema de alerta muy sencillo que nos indique cuando, en un minuto, hay una diferencia mayor de 20 acciones entre los contadores de las acciones ```Open``` y ```Close```.

** NOTA:** Recuerda re-ejecuta las celdas de la parte 3 para re-iniciar el flujo de datos. Sólo hay tres minutos en todo el dataset donde la condición descrita anteriormente se cumple.

In [36]:
from time import sleep

for i in range(10):
  <FILL IN>
  sleep(5)

### Ejercicio 3(c)


Ahora vamos a calcular la [media móvil simple](https://en.wikipedia.org/wiki/Moving_average) del número de acciones de los últimos 30 minutos. Tienes los detalles de como realizar este cálculo [aquí](https://en.wikipedia.org/wiki/Moving_average#Simple_moving_average).

In [38]:
for i in range(10):
  <FILL IN>
  sleep(5)

### Ejercicio 3(d)

Ahora vamos a calcular la [varianza](https://es.wikipedia.org/wiki/Varianza) en el número de acciones. Tienes los detalles de como calcular la varianza online [aquí](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Online_algorithm). Para esto, recupera usando una sentencia sql, la suma de las acciones del último minuto y ves actualizando el resultado de la varianza.

In [40]:
for i in range(50):
  <FILL IN>
  sleep(5)

### Ejercicio 3(d)

Responde las siguientes preguntas:

- ¿Qué es una arquitectura lambda? ¿Spark cumple con esta definición?
- ¿El código que has utilizado en la parte de streaming es reusable para procesado batch? ¿y viceversa?